In [2]:
!pip install -qU langchain langchain-openai langchain-community langchain-chroma langchain-text-splitters


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [4]:
print("데이터 로드를 시작합니다...")
# 1. 텍스트 파일 로드
loader = TextLoader("cyberpunk_shards.txt", encoding="utf-8")
documents = loader.load()
print(f"총 {len(documents)}개의 문서를 로드했습니다.")

print("텍스트 분할을 시작합니다...")
# 2. 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)
print(f"총 {len(docs)}개의 텍스트 조각(chunk)으로 분할했습니다.")

print("임베딩 및 벡터 DB 구축을 시작합니다...")
# 3. 임베딩 모델 정의
embed_model = OpenAIEmbeddings(model="text-embedding-3-small")

# 4. 벡터 DB 생성 (Chroma)
# "./cyberpunk_chroma_db" 폴더에 DB가 저장됩니다.
db = Chroma.from_documents(
    documents=docs, 
    embedding=embed_model, 
    persist_directory="./cyberpunk_chroma_db"
)

# 5. 검색기(Retriever) 생성
# RAG에서 '검색'을 담당할 부품입니다.
retriever = db.as_retriever()

print("🎉 벡터 DB 구축 완료. 검색기(retriever)가 준비되었습니다.")

데이터 로드를 시작합니다...
총 1개의 문서를 로드했습니다.
텍스트 분할을 시작합니다...
총 1281개의 텍스트 조각(chunk)으로 분할했습니다.
임베딩 및 벡터 DB 구축을 시작합니다...
🎉 벡터 DB 구축 완료. 검색기(retriever)가 준비되었습니다.


In [5]:
# 1. LLM 모델 정의 (gpt-4o-mini가 최신 가성비 모델입니다)
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 2. RAG 프롬프트 템플릿
# LLM에게 Context(샤드 내용)를 주며 질문에 답하라고 지시합니다.
# "모르면 모른다고 답하라"는 지시어가 환각(Hallucination)을 막는 핵심입니다.
template = """
당신은 '사이버펑크 2077' 세계관 전문가입니다.
제공된 Context(샤드 내용)만을 바탕으로 사용자의 질문에 답변해 주세요.
만약 Context에 질문과 관련된 내용이 없다면, "죄송합니다. 제가 아는 샤드 내용 중에는 해당 정보가 없습니다."라고 답변하세요.

Context:
{context}

Question:
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. RAG 체인(Chain) 구성
# LangChain의 파이프라인(LCEL) 문법입니다.
# {context: retriever, question: ...} -> 질문을 받아 context(샤드)를 검색
# | prompt                         -> 검색된 context와 질문을 프롬프트에 삽입
# | llm                            -> LLM에 전달
# | StrOutputParser()              -> LLM의 답변(AIMessage)을 텍스트로 변환
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG 챗봇 체인이 구성되었습니다.")

RAG 챗봇 체인이 구성되었습니다.


In [6]:
print("--- RAG 챗봇 테스트 시작 ---")

# --- 테스트 1: (RAG 질문) 샤드에만 있는 내용 ---
# (cyberpunk_shards.txt에 '학생의 일기' 내용이 실제로 있어야 합니다)
question1 = "'[24.06.77] YES!!!' 샤드 내용은 뭐야?"
answer1 = rag_chain.invoke(question1)
print(f"\n[질문 1] {question1}")
print(f"[답변 1] {answer1}")

# --- 테스트 2: (일반 상식) RAG가 통제되는지 확인 ---
# 이 질문은 샤드에 없을 것이므로, LLM이 아는 내용이라도 "모르겠다"고 답해야 정상입니다.
# (프롬프트에서 'Context'만을 바탕으로 답하라고 지시했기 때문)
question2 = "키아누 리브스가 연기한 캐릭터의 이름은?"
answer2 = rag_chain.invoke(question2)
print(f"\n[질문 2] {question2}")
print(f"[답변 2] {answer2}")

# --- 테스트 3: (최신 정보) 샤드에도, LLM도 모르는 내용 ---
question3 = "오늘 날짜(2025년 11월 15일)로 사이버펑크 2077의 새로운 패치 소식이 있어?"
answer3 = rag_chain.invoke(question3)
print(f"\n[질문 3] {question3}")
print(f"[답변 3] {answer3}")

--- RAG 챗봇 테스트 시작 ---

[질문 1] '[24.06.77] YES!!!' 샤드 내용은 뭐야?
[답변 1] '[24.06.77] YES!!!' 샤드 내용은 누군가에게 답장을 보내는 내용으로, 오랜 기다림에 대해 사과하며, 상대방의 희생을 갚고 싶다는 마음이 담겨 있습니다. 이 샤드는 주머니 속의 소중한 샤드가 자신이 얼마나 운이 좋은지를 상기시켜준다고 언급하며, 상대방이 이 메시지를 읽고 새로운 삶의 장을 함께 시작할 수 있기를 고대하고 있다는 내용입니다.

[질문 2] 키아누 리브스가 연기한 캐릭터의 이름은?
[답변 2] 죄송합니다. 제가 아는 샤드 내용 중에는 해당 정보가 없습니다.

[질문 3] 오늘 날짜(2025년 11월 15일)로 사이버펑크 2077의 새로운 패치 소식이 있어?
[답변 3] 죄송합니다. 제가 아는 샤드 내용 중에는 해당 정보가 없습니다.


In [7]:
question4 = "아라사카에 대해 알려줘"
answer4 = rag_chain.invoke(question4)
print(f"\n[질문 4] {question4}")
print(f"[답변 4] {answer4}")


[질문 4] 아라사카에 대해 알려줘
[답변 4] 아라사카는 제국을 형성한 가족으로, 그들의 이야기는 진정한 왕조를 배경으로 하고 있습니다. 아라사카 가족의 수장인 사부로 아라사카는 오랜 세월 동안 경영을 이어왔으며, 그의 은퇴와 함께 회사의 통제권이 그의 딸 한코 아라사카와 아들 요리노부 아라사카에게 넘어갈 것이라는 소문이 최근 몇 년 동안 돌고 있습니다. 이러한 소문이 진실을 포함하고 있는지 여부는 여전히 불확실합니다.
